# Evaluating machine learning models

Machine learning involves automatically learning how to compute functions from examples.  There are several ways that this process can go wrong, including:

0. Overfitting the training examples,
1. Optimizing for the wrong objective,
2. Starting with the wrong features, and
3. Data drift (which we'll treat in [another notebook](05-data-drift.ipynb)).

Let's look at some of these problems now.

In [1]:
import pandas as pd
import os.path

data = pd.read_parquet(os.path.join("data", "training.parquet"))

# Overfitting

The first concern we'd like to address is *overfitting*, in which we have a model whose performance on training examples is materially different from its performance in production.  We'll see that in action with a simple example.  First, let's choose some of our data as training examples:

In [2]:
overfit_training = data.sample(1000)
overfit_training.head(10)

,index,label,text
19387,19387,legitimate,What shall we do to rouse them? He engaged to ...
27180,7180,spam,I've been experimenting with Cajun food for qu...
10090,10090,legitimate,I admire your little establishment exceedingly...
26147,6147,spam,There was no residue at the bottom of the bag....
9644,9644,legitimate,Recently sent this to my son in Afghanistan he...
18476,18476,legitimate,I also add this to it. These are a great produ...
2667,2667,legitimate,He had met Mrs. Jennings at the door of the Co...
26069,6069,spam,This is the 1st K-cup coffee that I have ever ...
4642,4642,legitimate,"Mary had shewn herself disobliging to him, and..."
33591,13591,spam,These are every bit as good as those unroasted...


We'll "train" a very simple "model" from these examples:  we'll just memorize hashes of every example so we can look up whether a given example is legitimate or not.  We'll program defensively, too:  if we don't find an example in either set, we'll call it legitimate if it has an even number of characters.

In [3]:
class OverfittingSpamModel(object):
    def __init__(self):
        self.legit = set()
        self.spam = set()
    
    def fit(self, df):
        for tup in df.itertuples():
            if tup.label == "legitimate":
                self.legit.add(hash(tup.text))
            else:
                self.spam.add(hash(tup.text))
    
    def predict(self, text):
        h = hash(text)
        if h in self.legit:
            return "legitimate"
        elif h in self.spam:
            return "spam"
        else:
            return (len(text) % 2 == 0) and "legitimate" or "spam"

osm = OverfittingSpamModel()
osm.fit(overfit_training)

We can try this out with some of our training examples to see how well it works:

In [4]:
for row in overfit_training.sample(10).itertuples():
    print("text is '%s...': actual label is %s; predicted label is %s" % (row.text[0:20], row.label, osm.predict(row.text)))

text is 'The only difference ...': actual label is legitimate; predicted label is legitimate
text is 'Great stuff. I can't...': actual label is spam; predicted label is spam
text is 'She smiled, but made...': actual label is legitimate; predicted label is legitimate
text is 'The invitation was e...': actual label is legitimate; predicted label is legitimate
text is 'The young ladies arr...': actual label is legitimate; predicted label is legitimate
text is 'I could have no mode...': actual label is legitimate; predicted label is legitimate
text is 'This wonderful combi...': actual label is spam; predicted label is spam
text is 'Day! The house was c...': actual label is legitimate; predicted label is legitimate
text is 'I know it may sound ...': actual label is spam; predicted label is spam
text is 'The original Catalin...': actual label is spam; predicted label is spam


This model appears to work really well!  We can test it on the whole set of examples:

In [5]:
def model_accuracy(osm, df):
    correct = 0
    incorrect = 0
    for row in df.itertuples(): 
        if row.label == osm.predict(row.text):
            correct += 1
        else:
            incorrect += 1
    
    if correct + incorrect == 0:
        return 100
    
    return (float(correct) / float(correct + incorrect) * 100)

In [6]:
model_accuracy(osm, overfit_training)

100.0

Our model is enormously successful!  It has one hundred percent accuracy.  We probably expected this result, but it's always nice when things work out as you expected they would.  Let's see how well our model has generalized to data it _hasn't_ seen by testing it on the rest of our dataset (39,000 more examples).

In [7]:
model_accuracy(osm, data)

51.629999999999995

Uh oh!  It appears that our model is not much better than a coin toss once it's running on data it hasn't already seen.  If we had put this model in production, our application surely wouldn't have performed well.

We want a way to identify this problem before we put a model into production, and you've essentially seen it already:  when we train a model, we don't use all of the data we have available to us.  Instead, we divide our examples into distinct _training_ and _test_ sets, usually with about 70% of the examples in the former and 30% in the latter.  
The training algorithm only considers the examples in the training set.  After training our model, we can evaluate its performance on both the training set (which it saw during training) and the test set (which it didn't).  If the performance is materially different on the different sets, we know that we've overfit the data when training our model.

Next up, we'll deal with the question of what metrics we should use to evaluate our performance.  We used accuracy above, but is it always the best option?

# Evaluation metrics and types of error

Our training data set is _balanced_ between classes -- there are equal numbers of legitimate and spam documents.  But data in the real world are typically not balanced, and are often wildly unbalanced.  For example:

- The worldwide incidence of Rh-negative blood types is approximiately 6 percent;
- Between one and three percent of actual consumer payments transactions are fraudulent; and
- A rare disease may have an incidence rate on the order of one in ten thousand per year.

In cases like these, it would be possible to develop an accurate model that wouldn't produce meaningful results; for example:

- A blood type tester that always returned "Rh-positive" would be accurate roughly 94% of the time on a sufficiently diverse population;
- A fraud detector that always returned "not fradulent" would be accurate between 97-99% of the time -- until, that is, fraudsters determined that their charges would likely go through, increasing the rate of fraudulent charges; and
- A technique to screen for a very rare disease could be quite accurate by simply never identifying disease.

In many applications, we're not only interested in correctly identifying members of one class, we're interested in correctly identifying members of both classes.  We can capture this behavior by using better metrics than accuracy.

To learn about these metrics, let's start with an unbalanced data set, in which 90% of the messages are spam.

In [8]:
legit_sample = data[data.label == 'legitimate'].sample(2000)
spam_sample = data[data.label == 'spam'].sample(18000)
unbalanced = pd.DataFrame.append(legit_sample, spam_sample)

To avoid overfitting, we'll split the unbalanced data set into training and test sets, using functionality from scikit-learn.

In [9]:
from sklearn.model_selection import train_test_split
unbalanced_train, unbalanced_test = train_test_split(unbalanced, test_size=0.3)

We'll now create a simple model that should work pretty well for spam messages but not necessarily as well for legitimate ones.  

In [10]:
from collections import defaultdict
import re
    
class SensitiveSpamModel(object):
    
    def __init__(self):
        self.legit = set()
        self.spam = set()
    
    def fit(self, df):
        """ Train a model based on the most frequent unique 
            words in each class of documents """
        legit_words = defaultdict(lambda: 0)
        spam_words = defaultdict(lambda: 0)
        
        for tup in df.itertuples():
            target = spam_words
            if tup.label == "legitimate":
                target = legit_words
            for word in re.split(r"\W+", tup.text):
                if len(word) > 0:
                    target[word.lower()] += 1
        
        # remove words common to both classes
        for word in set(legit_words.keys()).intersection(set(spam_words.keys())):
            del legit_words[word]
            del spam_words[word]
        
        top_legit_words = sorted(legit_words.items(), key=lambda kv: kv[1], reverse=True)
        top_spam_words = sorted(spam_words.items(), key=lambda kv: kv[1], reverse=True)
        
        # store ten times as many words from the spam set
        self.legit = set([t[0] for t in top_legit_words[:100]])
        self.spam = set([t[0] for t in top_spam_words[:1000]])
            
    def predict(self, text):
        legit_score = 0
        spam_score = 0
        
        for word in re.split(r"\W+", text):
            w = word.lower()
            if word in self.legit:
                legit_score = legit_score + 1
            elif word in self.spam:
                spam_score = spam_score + 1
        
        # bias results towards spam in the event of ties
        return (legit_score > spam_score) and "legitimate" or "spam"

ssm = SensitiveSpamModel()
ssm.fit(unbalanced_train)

Let's check the accuracy on our training sample.

In [11]:
model_accuracy(ssm, unbalanced_train)

93.68571428571428

To make sure we've not overfit our training sample, let's check the accuracy on the test sample.

In [12]:
model_accuracy(ssm, unbalanced_test)

92.56666666666666

That's not quite as good as the results on our training sample, but it's still pretty decent (that is, it's better than just always returning "spam" would be given the balance of the classes).  

However, we get a different picture if we look at our model's performance on the balanced data.

In [13]:
model_accuracy(ssm, data)

62.975

And the accuracy is even worse if we look at a sample where the label balance is reversed (i.e., only 10% of documents are spam):

In [14]:
legit_sample = data[data.label == 'legitimate'].sample(900)
spam_sample = data[data.label == 'spam'].sample(100)
legit_biased = pd.DataFrame.append(legit_sample, spam_sample)
model_accuracy(ssm, legit_biased)

35.0

We'd like to understand the performance of our model with some metric that captures not only the overall accuracy but the accuracy for positive cases and the accuracy for negative cases.  That is, if we assume that our goal is to identify spam documents, we care about:

- *true positives*, which are spam documents that our model predicts as spam;
- *true negatives*, which are legitimate documents that our model predicts as legitimate;
- *false positives*, which are legitimate documents that our model predicts as spam; and
- *false negatives*, which are spam documents that our model predicts as legitimate

The proportions between these quantities can provide interesting metrics.  For example, the ratio of true positives to actual positives (that is, true positives + false negatives) is called *recall*, which indicates the percentage of spam documents we've selected.  The ratio of true positives to all predicted positives (that is, true positives + false positives) is called *precision*, which indicates the percentage of predicted spam documents that are actually spam.  Ideally, a good classifier would have both high precision and high recall, but in some applications either precision or recall is more important.

We can visualize the overall performance of a classifier with a *confusion matrix*, which plots actual labels in rows and predicted labels in columns.  The confusion matrix thus puts correct predictions along one diagonal and various kinds of incorrect predictions elsewhere.  Let's see one in action for our data.

In [15]:
from mlworkflows import plot

def true_and_predicted(df, model):
    return (df.label.values, [model.predict(txt) for txt in df.text.values])

df, chart = plot.binary_confusion_matrix(*true_and_predicted(unbalanced, ssm))


In [16]:
chart

alt.Chart(...)

In [17]:
df

,predicted,actual,raw_count,value
0,legitimate,legitimate,676,0.338000
1,legitimate,spam,1324,0.662000
2,spam,legitimate,6,0.000333
3,spam,spam,17994,0.999667


It should be clear that accuracy isn't always the best metric.  We're not merely interested in accuracy, but also in false positive rate and false negative rate.  Furthermore, in some cases, we care more about false negatives, and in some cases, we care more about false positives:  for example, in a disease screening application, false positives may be costly but false negatives may be deadly.  The [F<sub>β</sub>-score](https://en.wikipedia.org/wiki/F1_score) is a family of measures that allow us to ascribe relative importance to false positives and false negatives while evaluating a classifier.

If we want to weight false positives and false negatives equally, the formula looks like this:

In [18]:
def f1_score(actual_values, predicted_values, positive_label="spam"):
    actual_positives = float(len([v for v in actual_values if v == positive_label]))
    true_positives = float(len([t for t in zip(actual_values, predicted_values) if t[0] == t[1] == positive_label]))
    predicted_positives = float(len([v for v in predicted_values if v == positive_label]))
    
    precision = true_positives / predicted_positives
    recall = true_positives / actual_positives
    return 2 * ((precision * recall) / (precision + recall))

In [19]:
f1_score(*true_and_predicted(unbalanced, ssm))

0.9643603622916556

We can generalize this formula to emphasize false negatives or false positives.  The F<sub>β</sub>-score weights recall by β times as much as precision -- thus, if β is greater than one, false negatives will be weighted more heavily, but if β is less than one, false positives will be weighted more heavily.

In [20]:
def fbeta_score(actual_values, predicted_values, beta, positive_label="spam"):
    actual_positives = float(len([v for v in actual_values if v == positive_label]))
    true_positives = float(len([t for t in zip(actual_values, predicted_values) if t[0] == t[1] == positive_label]))
    predicted_positives = float(len([v for v in predicted_values if v == positive_label]))
    
    precision = true_positives / predicted_positives
    recall = true_positives / actual_positives
    return (1 + (beta * beta)) * ((precision * recall) / (((beta * beta) * precision) + recall))

In [21]:
fbeta_score(*true_and_predicted(unbalanced, ssm), 1)

0.9643603622916556

In [22]:
fbeta_score(*true_and_predicted(unbalanced, ssm), 2)

0.985238397687203

In [23]:
fbeta_score(*true_and_predicted(unbalanced, ssm), 0.5)

0.944348811822991

We've included these implementations to show you how simple the formula is, but for real applications, you'd want to use the [scikit-learn implementation of F<sub>1</sub>-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score) or [F<sub>β</sub>-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html#sklearn.metrics.fbeta_score).  The main takeaway from this section should be that the metrics you use to evaluate your models are important, and they can affect both how you train your models and how you monitor them in production.

# Problems with feature selection

Sometimes, the features we choose wind up not being that informative.  In [our previous notebook](01-vectors-and-visualization.ipynb), we saw an example of a very simple feature that exposed some structure in our data.  However, we won't always be so lucky as to land on a feature engineering approach that makes it so easy to separate kinds of examples.  Let's say that, instead of hashing <em>k</em>-shingles, we instead wanted to count the number of words with a given length.  We'll turn our training set into a set of labeled count vectors.

In [24]:
import numpy as np

def count_frequency(txt, MAX_LEN=7):
    """ returns an array of the number of words 
        with a given length for each length up to MAX_LEN """
    result = np.zeros(MAX_LEN + 1)
    for word in re.split(r"\W+", txt):
        idx = min(len(word), MAX_LEN)
        result[idx] += 1.0
    return result

As before, we'll use PCA to project our points into two dimensions and then plot them to see if there's any structure with our data.

In [25]:
plot.plot_pca(data, "text", func=count_frequency, x="x", y="y", color="label")

alt.Chart(...)

Zoom and pan to examine that plot more closely, if you haven't already.  Surprisingly, there is still some structure to our data, but the boundary between the classes is awfully large (and fuzzy).  Just to be sure, we could try using t-SNE to visualize these data as well.

In [26]:
np.random.seed(0xc0ffee)
plot.plot_tsne(data, "text", func=count_frequency, tsne_sample=800, x="x", y="y", color="label")

alt.Chart(...)

The structure in the t-SNE plot isn't particularly obvious, either.  However, some potential feature engineering approaches are even worse.  For example, encoding merely whether or not we've seen a word of a given length, so that `"Bob ate chips"` would be encoded as `[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...]` -- i.e., so that it would have a `1.0` in positions 3 and 5.

In [27]:
def count_existence(txt, MAX_LEN=15):
    """ returns an array of 1.0 if the input had a word with the given length 
        and 0.0 if the input did not, for each length up to MAX_LEN"""
    result = np.zeros(MAX_LEN + 1)
    for word in re.split(r"\W+", txt):
        idx = min(len(word), MAX_LEN)
        result[idx] = 1.0
    return result

plot.plot_pca(data, "text", func=count_existence, x="x", y="y", color="label")

alt.Chart(...)

As before, we can see if t-SNE gets us a more useful visualization of the structure of our data.

In [28]:
np.random.seed(0xc0ffee)
plot.plot_tsne(data, "text", func=count_existence, tsne_sample=800, x="x", y="y", color="label")

alt.Chart(...)

...but, also as before, it's not obvious how we'd separate these classes of document based on the features we have.

This kind of encoding can actually work quite well for some learning problems, but it should be clear that it doesn't work that well for this one.  In a situation like this, when visualizing your feature vectors shows a noisy or indistinguishable map, you may need to go back to the drawing board and choose a different feature engineering approach.

In the next part of the workshop, you get to choose between two different feature engineering approaches. You can proceed either to [this notebook, which shows an approach based on summaries of various properties of the documents](03-feature-engineering-summaries.ipynb) or [this notebook, which shows an approach based on which terms appear most frequently in documents](03-feature-engineering-tfidf.ipynb).